In [1]:
from IPython.display import clear_output
import openmdao.api as om
clear_output()
import importlib
import GUI_notebook
importlib.reload(GUI_notebook)
vb = GUI_notebook.init(In)
vb

Box(children=(Box(children=(Btn(children=['Copy cells'], color='blue lighten-1', height='35px', width='250px')…

In [2]:
# Import
import math
import numpy as np


In [3]:
# Init
Dpro_ref=11*.0254 # [m] diameter
Mpro_ref=0.53*0.0283 # [kg] mass
Tmot_ref=2.32  # [N.m] rated torque
Tmot_max_ref=85/70*Tmot_ref # [N.m] max torque
Rmot_ref=0.03  # [Ohm] resistance
Mmot_ref=0.575 # [kg] mass
Ktmot_ref=0.03 # [N.m/A] torque coefficient
Tfmot_ref=0.03 # [N.m] friction torque (zero load, nominal speed) 
Mbat_ref=.329 # [kg] mass
Cbat_ref= 3.400*3600 # [A.s] 
Vbat_ref=4*3.7 # [V] 
Imax_ref=170 # [A]
Mfra_ref=.347 #[kg] MK7 frame
Marm_ref=0.14#[kg] Mass of all arms
Pesc_ref=3108 # [W] Power
Vesc_ref=44.4 #[V]Voltage
Mesc_ref=.115 # [kg] Mass

In [4]:
#%% DRONE

#% SCENARIOS
# ---
M_total = k_os * M_pay # [kg] Estimation of the total mass (or equivalent weight of dynamic scenario)
F_pro_hov = M_total * (9.81) / N_pro # [N] Thrust per propeller for hover
F_pro_to = M_total * (9.81 + a_to) / N_pro # [N] Thrust per propeller for take-off


#% PROPELLER
# --- 

C_t = 4.27e-02 + 1.44e-01 * beta_pro  # Thrust coef with T=C_T.rho.n^2.D^4 - 0.8 for de-rating of APC catalog
C_p = -1.48e-03 + 9.72e-02 * beta_pro  # Power coef with P=C_p.rho.n^3.D^5

# Propeller selection with take-off scenario
D_pro = (F_pro_to / (C_t*rho_air*(ND_max/k_ND)**2.))**0.5 # [m] Propeller diameter
n_pro_to = ND_max / k_ND / D_pro # [Hz] Propeller speed 
Omega_pro_to = n_pro_to * 2*np.pi # [rad/s] Propeller speed

M_pro = M_pro_ref * (D_pro/D_pro_ref)**2. # [kg] Propeller mass

P_pro_to = C_p * rho_air * n_pro_to**3. * D_pro**5. # [W] Power per propeller
T_pro_to = P_pro_to / Omega_pro_to # [N*m] Propeller torque

# Propeller torque & speed for hover
n_pro_hov = np.sqrt(F_pro_hov/(C_t * rho_air *D_pro**4.)) # [Hz] hover speed
Omega_pro_hov = n_pro_hov * 2.*np.pi # [rad/s] Propeller speed

P_pro_hov = C_p * rho_air * n_pro_hov**3. * D_pro**5. # [W] Power per propeller
T_pro_hov = P_pro_hov / Omega_pro_hov # [N*m] Propeller torque       
U_bat_est = k_vb*1.84*P_pro_to**(0.36) # [V] battery voltage estimation


#% MOTOR
# --- 
T_nom_mot = k_mot * T_pro_hov   # [N*m] Motor nominal torque per propeller

M_mot = M_mot_ref * (T_nom_mot/T_nom_mot_ref)**(3./3.5) # [kg] Motor mass

# Selection with take-off speed
K_mot = U_bat_est / (k_speed_mot*Omega_pro_to) # [N*m/A] or [V/(rad/s)] Kt motor

R_mot = R_mot_ref * (T_nom_mot/T_nom_mot_ref)**(-5./3.5)*(K_mot/K_mot_ref)**2.  # [ohm] motor resistance
T_mot_fr = T_mot_fr_ref * (T_nom_mot/T_nom_mot_ref)**(3./3.5) # [N*m] Friction torque
T_max_mot = T_max_mot_ref * (T_nom_mot/T_nom_mot_ref)

# Hover current and voltage
I_mot_hov = (T_pro_hov+T_mot_fr) / K_mot # [A] Current of the motor per propeller
U_mot_hov = R_mot*I_mot_hov + Omega_pro_hov*K_mot # [V] Voltage of the motor per propeller
P_el_mot_hov = U_mot_hov*I_mot_hov # [W] Hover : electrical power

# Takeoff current and voltage
I_mot_to = (T_pro_to+T_mot_fr) / K_mot # [A] Current of the motor per propeller
U_mot_to = R_mot*I_mot_to + Omega_pro_to*K_mot # [V] Voltage of the motor per propeller
P_el_mot_to = U_mot_to*I_mot_to # [W] Takeoff : electrical power

#% BATTERY
# ---     
# Battery selection & scaling laws sized from hover

N_s_bat = U_bat_est/3.7 # Cell number, round (up value)
U_bat = 3.7 * N_s_bat # [V] Battery voltage

M_bat = k_mb * M_pay # [kg] Battery mass

# Hover --> autonomy
E_bat = E_bat_ref * M_bat/M_bat_ref*.8 # [J] Energy  of the battery (.8 coefficient because 80% use only of the total capacity)
C_bat = E_bat/U_bat # [A*s] Capacity  of the battery 
I_bat = (P_el_mot_hov * N_pro) /.95/U_bat # [A] Current of the battery

#% ESC
P_esc = P_el_mot_to * U_bat / U_mot_to # [W] power electronic power (corner power or apparent power)
M_esc = M_esc_ref * (P_esc/P_esc_ref) # [kg] Mass ESC
V_esc = 1.84 * P_esc**0.36 # [V] ESC voltage        

#% FRAME
# ---
alpha_sep = 2*np.pi / N_arm # [rad] interior angle separation between propellers
L_arm = D_pro / (2.*np.sin(alpha_sep/2.)) # [m] length of the arm

# Tube diameter & thickness
D_out_arm = (F_pro_to*N_pro_arm/sigma_max*L_arm*32./(np.pi*(1.-(1.-2.*k_D)**4.)))**(1/3)  # [m] outer diameter of the arm (hollow cylinder)
e_arm = k_D * D_out_arm # [m] thickness of the arm (hollow cylinder) 
D_in_arm = D_out_arm - e_arm # [m] inner diameter of the arm (hollow cylinder) 

# Mass
M_arm = np.pi/4. * (D_out_arm**2.-(D_out_arm-2.*e_arm)**2)*L_arm*rho_s # [kg] mass of the arm (x1) (beams only)  composite
M_frame = N_arm * M_arm / 0.4 # [kg] mass of the frame (40% of total mass is the arms)

#% OBJECTIVES
# ---
t_hov = C_bat/I_bat/60. # [min] Hover time 
M_total_real=(M_esc+M_pro+M_mot)*N_pro+M_pay+M_bat+M_frame  # [kg] Total mass

#% CONSTRAINTS
cons_1 = M_total-M_total_real
cons_2 = U_bat-U_mot_to
cons_3 = T_max_mot-T_pro_to
cons_4 = U_bat-V_esc
cons_5 = V_esc-U_mot_to
cons_6 = MTOW-M_total_real
cons_7 = t_hov-t_hov_spec

NameError: name 'k_os' is not defined

In [ ]:
# Init
Dpro_ref=11*.0254 # [m] diameter
Mpro_ref=0.53*0.0283 # [kg] mass
Tmot_ref=2.32  # [N.m] rated torque
Tmot_max_ref=85/70*Tmot_ref # [N.m] max torque
Rmot_ref=0.03  # [Ohm] resistance
Mmot_ref=0.575 # [kg] mass
Ktmot_ref=0.03 # [N.m/A] torque coefficient
Tfmot_ref=0.03 # [N.m] friction torque (zero load, nominal speed) 
Mbat_ref=.329 # [kg] mass
Cbat_ref= 3.400*3600 # [A.s] 
Vbat_ref=4*3.7 # [V] 
Imax_ref=170 # [A]
Mfra_ref=.347 #[kg] MK7 frame
Marm_ref=0.14#[kg] Mass of all arms
Pesc_ref=3108 # [W] Power
Vesc_ref=44.4 #[V]Voltage
Mesc_ref=.115 # [kg] Mass